In [2]:
! module load python
! pip install --user dask[complete]==2022.7.0 dask-jobqueue==0.7.4 dask-ml==2022.5.27
! pip install --user --force-reinstall --upgrade jupyter

  Using cached dask-2022.7.0-py3-none-any.whl.metadata (3.5 kB)
Using cached dask-2022.7.0-py3-none-any.whl (1.1 MB)
  Attempting uninstall: dask
    Found existing installation: dask 2023.3.2
    Uninstalling dask-2023.3.2:
      Successfully uninstalled dask-2023.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask-expr 1.1.10 requires dask==2024.8.0, but you have dask 2022.7.0 which is incompatible.
dask-expr 1.1.10 requires pandas>=2, but you have pandas 1.5.3 which is incompatible.
  Using cached jupyter-1.1.1-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached jupyter_console-6.6.3-py3-none-any.whl.metadata (5.8 kB)
  Using cached nbconvert-7.16.6-py3-none-any.whl.metadata (8.5 kB)
  Using cached ipykernel-6.29.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached ipywidgets-8.1.7-py3-none-any.whl.metadata (2.4 kB)
  Using cached comm-0.2.2-py3-none-a

In [ ]:
import dask.dataframe as dd
from dask import delayed, compute
from dask_jobqueue import SLURMCluster
from dask.distributed import Client
import dask
import pandas as pd
import re
import dask.bag as db
import time
from numba.pycc import CC

def extract_word_pairs(text, group_keywords):
    pairs = []
    try:
        if "Here is" in text or "Here's" in text:
            if ':' in text:
                text = text.split(':', 1)[1]
            else:
                lines = text.split('\n')
                for i, line in enumerate(lines):
                    if '-' in line and any(word in line.lower() for word in group_keywords):
                        text = '\n'.join(lines[i:])
                        break
           
        lines = text.strip().split('\n')
        for line in lines:
            if not line.strip():
                continue

            line = re.sub(r'^(\d+[\.\)\s]*|-\s*)', '', line.strip())

            if '-' in line:
                parts = [p.strip().lower() for p in line.split('-', 1)]
            else:
                parts = [w.strip().lower() for w in line.split()]

      
            if len(parts) == 2 and all(parts):
                pairs.append(tuple(parts))

    except Exception as e:
        print(f"Error processing text: {e}")
        print(f"Problematic text: {text}")
    return pairs

# Use Numba to compile this same function in a module named `ds`
cc = CC('ds')
@cc.export('d_score', 'f8(f8, f8, f8, f8)')
def d_score(a, b, c, d):
    # Manually count
    a = sum(1 for g, v in subset if g == 'stigma' and v == 'negative')
    b = sum(1 for g, v in subset if g == 'stigma' and v == 'positive')
    c = sum(1 for g, v in subset if g == 'default' and v == 'negative')
    d = sum(1 for g, v in subset if g == 'default' and v == 'positive')

    if (a + b == 0) or (c + d == 0):
        return 0.0
    return (a / (a + b)) + (d / (c + d)) - 1
cc.compile()


/home/yining11/ipykernel_1531131/2727287843.py:10: NumbaPendingDeprecationWarning: The 'pycc' module is pending deprecation. Replacement technology is being developed.

Pending Deprecation in Numba 0.57.0. For more information please see: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-the-numba-pycc-module
  from numba.pycc import CC


In [ ]:
def process_iat_data_parallel(input_file, label_file, output_file):
    """
    input_file: str, path to CSV with IAT data (column: 'iat')
    label_file_dict: dict, maps dataset name to label file path, e.g. {'sexuality': 'sexuality.csv'}
    dataset_name: str, which dataset to use (key in label_file_dict)
    output_file: str, where to save processed CSV
    """

    label_df = pd.read_csv(label_file)
    
    # 2. Define A/B: groups (e.g., straight vs gay)
    A = [str(w).lower() for w in label_df['A'].dropna().unique()]
    B = [str(w).lower() for w in label_df['B'].dropna().unique()]
    E = []
    F = []

    # 3. Define E/F: valence (positive vs negative)
    for dataset_name in label_df['dataset'].dropna().unique():
        subset = label_df[label_df['dataset'] == dataset_name].copy()
        mid = len(subset) // 2
        E = subset.iloc[:mid][['C', 'D']].values.flatten()
        F = subset.iloc[mid:][['C', 'D']].values.flatten()
        E = [str(w).strip().lower() for w in subset.iloc[:mid][['C', 'D']].values.flatten() if pd.notna(w)]
        F = [str(w).strip().lower() for w in subset.iloc[mid:][['C', 'D']].values.flatten() if pd.notna(w)]

    E = list(set(E))
    F = list(set(F))

    # 4. Load data
    dfs = dd.read_csv(input_file)

    # Segment response column
    pattern = r'Task 1:\s*([\s\S]*?)Task 2:\s*([\s\S]*?)Task 3:\s*([\s\S]*?)Task 4:\s*([\s\S]*)'
    dfs[['iat', 'profile', 'decision','report']] = dfs['response'].str.extract(pattern)

    # Map report to score
    mapping_dict = {
        "Strongly disagree": -2,
        "Disagree": -1,
        "Neutral": 0,
        "Agree": 1,
        "Strongly agree": 2
    }
    dfs['report_score'] = dfs['report'].map(mapping_dict).fillna(0)

    # Convert to pandas once
    pdf = dfs.compute()
    iat_list = list(enumerate(pdf['iat']))
    
    group_keywords = A + B 

    def process_entry(entry):
        i, text = entry
        pairs = extract_word_pairs(str(text),group_keywords)
        valence, group = zip(*pairs) if pairs else ([], [])
        group_label = ['default' if g in A else 'stigma' if g in B else 'error' for g in group]
        valence_label = ['positive' if v in E else 'negative' if v in F else 'error' for v in valence]
        subset = [(g, v) for g, v in zip(group_label, valence_label) if 'error' not in (g, v)]
        
        # Manually count
        a = sum(1 for g, v in subset if g == 'stigma' and v == 'negative')
        b = sum(1 for g, v in subset if g == 'stigma' and v == 'positive')
        c = sum(1 for g, v in subset if g == 'default' and v == 'negative')
        d = sum(1 for g, v in subset if g == 'default' and v == 'positive')

        score = d_score(a, b, c, d)
        pair_text = '\n'.join([f"{v} - {g}" for v, g in zip(valence, group)])
        return (i, score, pair_text)

    bag = db.from_sequence(iat_list, npartitions=16)
    results = bag.map(process_entry).compute()

    indices, scores, pairs_text = zip(*results)
    pdf['iat_bias'] = scores
    pdf['word_pairs'] = pairs_text
    pdf.to_csv(output_file, index=False)
    return pdf


In [8]:
if __name__ == "__main__":
    start_time = time.time()
    # Set scheduler config if needed
    dask.config.set(scheduler='distributed')  # default when using Client

    # Start Dask SLURM cluster
    cluster = SLURMCluster(
        queue='caslake',
        cores=10,
        memory='40GB',
        processes=10,
        walltime='01:00:00',
        interface='ib0',
        job_extra=['--account=macs30123']
    )
    cluster.scale(jobs=1)  # adjust if needed
    client = Client(cluster)

    print("✅ Dask cluster started. Dashboard:", client.dashboard_link)

    # Define your file paths
    DRIVE_BASE_PATH = '/home/yining11/Documents/final_project/data'
    input_file = f"{DRIVE_BASE_PATH}/llm_outputs.csv"
    label_file = f"{DRIVE_BASE_PATH}/iat_stimuli_synonym.csv"
    output_file = f"{DRIVE_BASE_PATH}/response_clean.csv"

    # Run parallel IAT processing
    df_result = process_iat_data_parallel(input_file, label_file, output_file)
    end_time = time.time()
    print("✅ Processing complete. Output saved to:", output_file)
    time_consume = end_time - start_time
    print('time used', time_consume)


/home/yining11/.local/lib/python3.9/site-packages/distributed/node.py:179: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 38179 instead
  warnings.warn(


✅ Dask cluster started. Dashboard: http://172.25.0.65:38179/status


✅ Processing complete. Output saved to: /home/yining11/Documents/final_project/data/response_clean.csv
time used 18.8993923664093
